In [ ]:
import os
job_class = os.getenv("DKUBE_JOB_CLASS")
if not job_class:
    !{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import json
##
import kfp
from kfp import components
from dkube.sdk.api import DkubeApi
import random, string
from termcolor import colored

In [ ]:
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
## These fields must be modified to allow the file to run based on your repo names
## After the files are modified, run all of the cells to execute

training_program = "training" # Change this to identify your Code Repo name
model = 'insurance' # Change this to identify your Model Repo name

In [ ]:
## These fields are specific to this example, and should not be modified

image = "ocdr/dkube-datascience-tf-cpu:v2.0.0-17"
serving_image = "ocdr/tensorflowserver:2.0.0"
training_script = "python insurance/training.py"
transformer_code='insurance/transformer.py'
user = os.getenv('USERNAME')
framework = "tensorflow"
f_version = "2.0.0"
output_mount_point = "/opt/dkube/out"

## Project ID, make sure IDE is created under the same project where the expriment is suppose to run
## Or else provide it manually in the project_id variable.
project_id = os.environ.get("DKUBE_PROJECT_ID")

run_id = 0


In [ ]:
@kfp.dsl.pipeline(
    name='dkube-insurance-pl',
    description='sample insurance pipeline'
)
def insurance_pipeline(token, tags, deployment_name):
    
    train       = dkube_training_op(token, json.dumps({"image": image}),
                                    tags=tags,
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    outputs=json.dumps([model]),
                                    output_mounts=json.dumps([output_mount_point]))
    
    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    name=deployment_name,
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program,
                                    transformer_code=transformer_code,
                                    production="true").after(train)

## Create a run

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
api = DkubeApi(token=token)

In [ ]:
res = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))

if project_id:
    tags = json.dumps([f"project:{project_id}"])
    project_name = os.environ.get("DKUBE_PROJECT_NAME")
    run_name = f"[{project_name}] Insurance Run{run_id}"
    experiment = f"[{project_name}] experiment"
    deployment_name = f"{os.getenv('DKUBE_PROJECT_NAME', 'insurance')}-{os.getenv('DKUBE_USER_LOGIN_NAME')}-%s"%res
else:
    tags = []
    run_name = f"Insurance Run {run_id}"
    experiment = "default"
    deployment_name = f"{os.getenv('DKUBE_USER_LOGIN_NAME')}-%s"%res

In [ ]:
deployment_id = api.get_deployment_id(name=deployment_name)
if not deployment_id:
    print(f"On pipeline completion the deployment {colored(deployment_name, 'green', attrs=['bold'])} will be created")
    
    client = kfp.Client(existing_token=token)
    client.create_run_from_pipeline_func(insurance_pipeline, run_name=run_name, experiment_name=experiment,
                                         arguments={"token":token, "tags":tags,
                                                    'deployment_name':deployment_name}
                                        )
    run_id += 1
    pl_config = {"DEPLOYMENT_NAME":deployment_name}
    %store pl_config

else:
    print("Deployment Already Existing, skipping create, try running the cells again")